In [107]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [108]:

data = pd.read_csv('chapter6/CH06PR18.txt',  header=None, sep='\s+')

In [109]:
data.rename({0:"Y", 1: "X_1", 2: "X_2", 3: "X_3", 4:"X_4"}, axis='columns', inplace=True)

In [110]:
data.head()

,Y,X_1,X_2,X_3,X_4
0,13.5000,1,5.0200,0.1400,123000
1,12.0000,14,8.1900,0.2700,104079
2,10.5000,16,3.0000,0.0000,39998
3,15.0000,4,10.7000,0.0500,57112
4,14.0000,11,8.9700,0.0700,60000


In [111]:
def get_ssr(model):
    return np.sum((model.fittedvalues - np.mean(model.model.endog))**2)

# Fit models in sequence to get Type I SS
model4 = smf.ols('Y ~ X_4', data=data).fit()
model14 = smf.ols('Y ~ X_4 + X_1', data=data).fit()
model124 = smf.ols('Y ~ X_4 + X_1 + X_2', data=data).fit()
model1234 = smf.ols('Y ~ X_4 + X_1 + X_2 + X_3', data=data).fit()

# Calculate sequential sums of squares
ssr_x4 = get_ssr(model4)
ssr_x1_given_x4 = get_ssr(model14) - get_ssr(model4)
ssr_x2_given_x14 = get_ssr(model124) - get_ssr(model14)
ssr_x3_given_x124 = get_ssr(model1234) - get_ssr(model124)

# Calculate total regression SS
ssr_total = ssr_x4 + ssr_x1_given_x4 + ssr_x2_given_x14 + ssr_x3_given_x124

# Calculate error and total SS
sse = np.sum(model1234.resid**2)
sst = np.sum((data['Y'] - np.mean(data['Y']))**2)

# Calculate degrees of freedom
n = len(data)
df_regression = 4    # total number of predictors
df_predictors = 1    # df for each predictor
df_error = n - 5     # n - number of parameters (including intercept)
df_total = n - 1

# Calculate Mean Squares
ms_regression = ssr_total / df_regression
ms_x4 = ssr_x4 / df_predictors
ms_x1 = ssr_x1_given_x4 / df_predictors
ms_x2 = ssr_x2_given_x14 / df_predictors
ms_x3 = ssr_x3_given_x124 / df_predictors
ms_error = sse / df_error


anova_table = pd.DataFrame({
    'Source': ['Regression', 'X_4', 'X_1|X_4', 'X_2|X_1,X_4', 'X_3|X_1,X_2,X_4', 'Error', 'Total'],
    'df': [df_regression, 1, 1, 1, 1, df_error, df_total],
    'SS': [ssr_total, ssr_x4, ssr_x1_given_x4, ssr_x2_given_x14, ssr_x3_given_x124, sse, sst],
    'MS': [ms_regression, ms_x4, ms_x1, ms_x2, ms_x3, ms_error, np.nan],

})

pd.set_option('display.float_format', lambda x: '%.4f' % x)
print("\nAnalysis of Variance Table:")
print(anova_table)

r_squared = model1234.rsquared
print(f"\nR-squared (full model): {r_squared:.4f}")


Analysis of Variance Table:
            Source  df       SS      MS
0       Regression   4 138.3269 34.5817
1              X_4   1  67.7751 67.7751
2          X_1|X_4   1  42.2746 42.2746
3      X_2|X_1,X_4   1  27.8575 27.8575
4  X_3|X_1,X_2,X_4   1   0.4197  0.4197
5            Error  76  98.2306  1.2925
6            Total  80 236.5575     NaN

R-squared (full model): 0.5847


In [112]:
import pandas as pd
import numpy as np
from scipy import stats

# We already have ssr_x3_given_x124 and ms_error from previous calculations
# Let's calculate the F* test statistic
f_star = ssr_x3_given_x124 / ms_error

# Degrees of freedom
df_numerator = 1  # one variable being tested
df_denominator = df_error  # from full model

# Calculate p-value
p_value = 1 - stats.f.cdf(f_star, df_numerator, df_denominator)

print(f"F* test statistic: {f_star:.4f}")
print(f"P-value: {p_value:.4f}")
alpha = 0.01
f_critical = stats.f.ppf(1 - alpha, df_numerator, df_denominator)
print(f"F critical test statistic: {f_critical:.4f}")
# Test decision
alpha = 0.01
print("\nHypothesis Test:")
print("H₀: β₃ = 0 (X₃ can be dropped)")
print("H₁: β₃ ≠ 0 (X₃ should be retained)")
print(f"Decision rule: Reject H₀ if p-value < {alpha}")
print(f"Conclusion: {'Reject' if p_value < alpha else 'Fail to reject'} H₀")

F* test statistic: 0.3248
P-value: 0.5704
F critical test statistic: 6.9806

Hypothesis Test:
H₀: β₃ = 0 (X₃ can be dropped)
H₁: β₃ ≠ 0 (X₃ should be retained)
Decision rule: Reject H₀ if p-value < 0.01
Conclusion: Fail to reject H₀


In [113]:
import pandas as pd
import numpy as np
from scipy import stats

# Calculate numerator (extra sum of squares for both X₂ and X₃ given X₁ and X₄)
ssr_x23_given_x14 = ssr_x2_given_x14 + ssr_x3_given_x124

# Calculate F* following equation 7.18
f_star = (ssr_x23_given_x14 / 2) / ms_error

# Degrees of freedom
df_numerator = 2  # testing two coefficients (β₂ and β₃)
df_denominator = df_error  # n - 5 (full model with 4 predictors plus intercept)

# Calculate p-value
p_value = 1 - stats.f.cdf(f_star, df_numerator, df_denominator)
alpha = 0.01
# Calculate critical F-value
f_critical = stats.f.ppf(1 - alpha, df_numerator, df_denominator)

print(f"F* test statistic: {f_star:.4f}")
print(f"Critical F-value (α = 0.01): {f_critical:.4f}")
print(f"P-value: {p_value:.4f}")

print("\nHypothesis Test:")
print("H₀: β₂ = β₃ = 0 (X₂ and X₃ can be dropped while keeping X₁ and X₄)")
print("H₁: At least one of β₂ or β₃ ≠ 0")
print(f"Decision rule: Reject H₀ if F* > {f_critical:.4f}")
print(f"Conclusion: {'Reject' if f_star > f_critical else 'Fail to reject'} H₀")

F* test statistic: 10.9389
Critical F-value (α = 0.01): 4.8958
P-value: 0.0001

Hypothesis Test:
H₀: β₂ = β₃ = 0 (X₂ and X₃ can be dropped while keeping X₁ and X₄)
H₁: At least one of β₂ or β₃ ≠ 0
Decision rule: Reject H₀ if F* > 4.8958
Conclusion: Reject H₀


In [106]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from scipy import stats

# Fit the reduced model (only X₁ and X₄)
model_reduced = ols('Y ~ X_1 + X_4', data=data).fit()

# Fit the full model (all variables)
model_full = ols('Y ~ X_1 + X_2 + X_3 + X_4', data=data).fit()

# Calculate SSE for both models
sse_reduced = np.sum(model_reduced.resid**2)
sse_full = np.sum(model_full.resid**2)

# Calculate the F* test statistic
df_numerator = 2  # number of variables being tested (X₂ and X₃)
df_denominator = len(data) - 5  # n - number of parameters in full model
df_error = df_denominator

# Extra sum of squares for X₂ and X₃ together
ssr_x23 = sse_reduced - sse_full

# Mean square error from full model
mse_full = sse_full / df_error

# Calculate F* statistic
f_star = (ssr_x23 / df_numerator) / mse_full

# Calculate p-value
p_value = 1 - stats.f.cdf(f_star, df_numerator, df_denominator)

print("Test for dropping X₂ and X₃:")
print(f"F* test statistic: {f_star:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"\nDegrees of freedom:")
print(f"Numerator (df₁): {df_numerator}")
print(f"Denominator (df₂): {df_denominator}")

print("\nHypothesis Test:")
print("H₀: β₂ = β₃ = 0 (both X₂ and X₃ can be dropped)")
print("H₁: At least one of β₂ or β₃ ≠ 0 (at least one should be retained)")
print(f"Decision rule: Reject H₀ if p-value < 0.01")
print(f"Conclusion: {'Reject' if p_value < 0.01 else 'Fail to reject'} H₀")

# Show the extra sum of squares components
print(f"\nExtra sum of squares removed: {ssr_x23:.4f}")
print(f"Mean Square Error (full model): {mse_full:.4f}")

Test for dropping X₂ and X₃:
F* test statistic: 10.9389
P-value: 0.0001

Degrees of freedom:
Numerator (df₁): 2
Denominator (df₂): 76

Hypothesis Test:
H₀: β₂ = β₃ = 0 (both X₂ and X₃ can be dropped)
H₁: At least one of β₂ or β₃ ≠ 0 (at least one should be retained)
Decision rule: Reject H₀ if p-value < 0.01
Conclusion: Reject H₀

Extra sum of squares removed: 28.2772
Mean Square Error (full model): 1.2925


In [115]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from scipy import stats

# Fit the full (unrestricted) model
model_full = ols('Y ~ X_1 + X_2 + X_3 + X_4', data=data).fit()

# Calculate SSE for full model
sse_full = np.sum(model_full.resid**2)

# Create restricted model variables
data['X1_restricted'] = -0.1 * data['X_1']  # Fixed coefficient
data['X2_restricted'] = 0.4 * data['X_2']   # Fixed coefficient
y_adjusted = data['Y'] - data['X1_restricted'] - data['X2_restricted']

# Fit restricted model for remaining parameters
model_restricted = ols('y_adjusted ~ X_3 + X_4', data=data).fit()
sse_restricted = np.sum(model_restricted.resid**2)

# Calculate degrees of freedom
n = len(data)
df_numerator = 2  # number of restrictions
df_denominator = n - 5  # n - number of parameters in full model

# Calculate F* statistic
f_star = ((sse_restricted - sse_full) / df_numerator) / (sse_full / df_denominator)

# Calculate critical F-value
alpha = 0.01
f_critical = stats.f.ppf(1 - alpha, df_numerator, df_denominator)

# Calculate p-value
p_value = 1 - stats.f.cdf(f_star, df_numerator, df_denominator)

# Print results
print("Test for β₁ = -0.1 and β₂ = 0.4:")
print(f"F* test statistic: {f_star:.4f}")
print(f"Critical F-value: {f_critical:.4f}")
print(f"P-value: {p_value:.4f}")

print("\nModels:")
print("Full model: Y = β₀ + β₁X₁ + β₂X₂ + β₃X₃ + β₄X₄")
print("Restricted model: Y = β₀ - 0.1X₁ + 0.4X₂ + β₃X₃ + β₄X₄")

print("\nHypothesis Test:")
print("H₀: β₁ = -0.1 and β₂ = 0.4")
print("H₁: β₁ ≠ -0.1 and/or β₂ ≠ 0.4")

print(f"\nDecision rules:")
print(f"Reject H₀ if p-value < {alpha}")
print(f"Reject H₀ if F* > {f_critical:.4f}")
print(f"\nConclusion: {'Reject' if p_value < alpha else 'Fail to reject'} H₀")

# Show the sum of squares
print(f"\nSSE (full model): {sse_full:.4f}")
print(f"SSE (restricted model): {sse_restricted:.4f}")
print(f"Difference in SSE: {(sse_restricted - sse_full):.4f}")

# Optional: Display model summaries
print("\nFull Model Summary:")
print(model_full.summary().tables[1])

print("\nRestricted Model Summary (for X₃ and X₄):")
print(model_restricted.summary().tables[1])

Test for β₁ = -0.1 and β₂ = 0.4:
F* test statistic: 4.6076
Critical F-value: 4.8958
P-value: 0.0129

Models:
Full model: Y = β₀ + β₁X₁ + β₂X₂ + β₃X₃ + β₄X₄
Restricted model: Y = β₀ - 0.1X₁ + 0.4X₂ + β₃X₃ + β₄X₄

Hypothesis Test:
H₀: β₁ = -0.1 and β₂ = 0.4
H₁: β₁ ≠ -0.1 and/or β₂ ≠ 0.4

Decision rules:
Reject H₀ if p-value < 0.01
Reject H₀ if F* > 4.8958

Conclusion: Fail to reject H₀

SSE (full model): 98.2306
SSE (restricted model): 110.1414
Difference in SSE: 11.9108

Full Model Summary:
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.2006      0.578     21.110      0.000      11.049      13.352
X_1           -0.1420      0.021     -6.655      0.000      -0.185      -0.100
X_2            0.2820      0.063      4.464      0.000       0.156       0.408
X_3            0.6193      1.087      0.570      0.570      -1.545       2.784
X_4         7.924e-06   1.38e-0

In [57]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols

# Read and prepare the data
data = pd.read_csv('chapter6/CH06PR18.txt', header=None, sep='\s+')
data.columns = ["Y", "X_1", "X_2", "X_3", "X_4"]

# Function to apply correlation transformation (standardization)
def correlation_transform(x):
    n = len(x)
    return (x - x.mean()) / (x.std() * np.sqrt(n-1))

# Transform all variables
data_transformed = pd.DataFrame()
data_transformed['Y_star'] = correlation_transform(data['Y'])
data_transformed['X1_star'] = correlation_transform(data['X_1'])
data_transformed['X2_star'] = correlation_transform(data['X_2'])
data_transformed['X3_star'] = correlation_transform(data['X_3'])
data_transformed['X4_star'] = correlation_transform(data['X_4'])

# Fit the standardized regression model
model_standardized = ols('Y_star ~ X1_star + X2_star + X3_star + X4_star', data=data_transformed).fit()

# Print results
print("Standardized Regression Results:")
print("\nStandardized Coefficients:")
for name, value in model_standardized.params.items():
    print(f"{name}: {value:.4f}")

print("\nModel Summary:")
print(f"R-squared: {model_standardized.rsquared:.4f}")
print("\nStandard Errors:")
print(model_standardized.bse)

# Verify standardization
print("\nVerification of standardization:")
print("\nMeans (should be close to 0):")
for column in data_transformed.columns:
    print(f"{column}: {data_transformed[column].mean():.10f}")

print("\nStandard Deviations (should be close to 1/sqrt(n-1)):")
expected_sd = 1/np.sqrt(len(data)-1)
for column in data_transformed.columns:
    print(f"{column}: {data_transformed[column].std():.10f} (Expected: {expected_sd:.10f})")

Standardized Regression Results:

Standardized Coefficients:
Intercept: -0.0000
X1_star: -0.5479
X2_star: 0.4236
X3_star: 0.0485
X4_star: 0.5028

Model Summary:
R-squared: 0.5847

Standard Errors:
Intercept   0.0082
X1_star     0.0823
X2_star     0.0949
X3_star     0.0850
X4_star     0.0879
dtype: float64

Verification of standardization:

Means (should be close to 0):
Y_star: -0.0000000000
X1_star: -0.0000000000
X2_star: 0.0000000000
X3_star: 0.0000000000
X4_star: 0.0000000000

Standard Deviations (should be close to 1/sqrt(n-1)):
Y_star: 0.1118033989 (Expected: 0.1118033989)
X1_star: 0.1118033989 (Expected: 0.1118033989)
X2_star: 0.1118033989 (Expected: 0.1118033989)
X3_star: 0.1118033989 (Expected: 0.1118033989)
X4_star: 0.1118033989 (Expected: 0.1118033989)


In [116]:
model_standardized.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Y_star   R-squared:                       0.585
Model:                            OLS   Adj. R-squared:                  0.563
Method:                 Least Squares   F-statistic:                     26.76
Date:                Wed, 19 Feb 2025   Prob (F-statistic):           7.27e-14
Time:                        21:05:52   Log-Likelihood:                 98.636
No. Observations:                  81   AIC:                            -187.3
Df Residuals:                      76   BIC:                            -175.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.645e-17      0.008  -3.22e-15      1.000      -0.016       0.016
X1_star       -0.5479      0.082     -6.655      0.000      -0.712      -0.384
X2_star        0.4236      0.095      4.464      0.000       0.235       0.613
X3_star        0.0485      0.085      0.570      0.570      -0.121       0.218
X4_star        0.5028      0.088      5.722      0.000       0.328       0.678
==============================================================================
Omnibus:                        1.922   Durbin-Watson:                   1.580
Prob(Omnibus):                  0.383   Jarque-Bera (JB):                1.301
Skew:                           0.148   Prob(JB):                        0.522
Kurtosis:                       3.545   Cond. No.                         14.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [58]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols

# Read and prepare the data
data = pd.read_csv('chapter6/CH06PR18.txt', header=None, sep='\s+')
data.columns = ["Y", "X_1", "X_2", "X_3", "X_4"]

# Calculate standard deviations for original variables
sy = data['Y'].std()
sx1 = data['X_1'].std()
sx2 = data['X_2'].std()
sx3 = data['X_3'].std()
sx4 = data['X_4'].std()

# Calculate means for original variables
y_mean = data['Y'].mean()
x1_mean = data['X_1'].mean()
x2_mean = data['X_2'].mean()
x3_mean = data['X_3'].mean()
x4_mean = data['X_4'].mean()

# Function to apply correlation transformation
def correlation_transform(x):
    n = len(x)
    return (x - x.mean()) / (x.std() * np.sqrt(n-1))

# Transform variables
data_transformed = pd.DataFrame()
data_transformed['Y_star'] = correlation_transform(data['Y'])
data_transformed['X1_star'] = correlation_transform(data['X_1'])
data_transformed['X2_star'] = correlation_transform(data['X_2'])
data_transformed['X3_star'] = correlation_transform(data['X_3'])
data_transformed['X4_star'] = correlation_transform(data['X_4'])

# Fit standardized model
model_standardized = ols('Y_star ~ X1_star + X2_star + X3_star + X4_star', data=data_transformed).fit()
b_star = model_standardized.params[1:]  # Exclude intercept

# Transform coefficients back to original scale using equation 7.53a
b1 = (sy/sx1) * b_star['X1_star']
b2 = (sy/sx2) * b_star['X2_star']
b3 = (sy/sx3) * b_star['X3_star']
b4 = (sy/sx4) * b_star['X4_star']

# Calculate b0 using equation 7.53b
b0 = y_mean - (b1*x1_mean + b2*x2_mean + b3*x3_mean + b4*x4_mean)

# Fit original model for verification
model_original = ols('Y ~ X_1 + X_2 + X_3 + X_4', data=data).fit()

print("Transformed Coefficients (from standardized back to original):")
print(f"b0 (intercept): {b0:.4f}")
print(f"b1: {b1:.4f}")
print(f"b2: {b2:.4f}")
print(f"b3: {b3:.4f}")
print(f"b4: {b4:.4f}")

print("\nOriginal Model Coefficients (for verification):")
print(model_original.params)

print("\nVerification - Difference between transformed and original coefficients:")
print(f"b0 diff: {abs(b0 - model_original.params[0]):.10f}")
print(f"b1 diff: {abs(b1 - model_original.params[1]):.10f}")
print(f"b2 diff: {abs(b2 - model_original.params[2]):.10f}")
print(f"b3 diff: {abs(b3 - model_original.params[3]):.10f}")
print(f"b4 diff: {abs(b4 - model_original.params[4]):.10f}")

Transformed Coefficients (from standardized back to original):
b0 (intercept): 12.2006
b1: -0.1420
b2: 0.2820
b3: 0.6193
b4: 0.0000

Original Model Coefficients (for verification):
Intercept   12.2006
X_1         -0.1420
X_2          0.2820
X_3          0.6193
X_4          0.0000
dtype: float64

Verification - Difference between transformed and original coefficients:
b0 diff: 0.0000000000
b1 diff: 0.0000000000
b2 diff: 0.0000000000
b3 diff: 0.0000000000
b4 diff: 0.0000000000


/var/folders/52/y8mfvm1n6vj12ntkl_p9hqw80000gn/T/ipykernel_48711/1253503542.py:63: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"b0 diff: {abs(b0 - model_original.params[0]):.10f}")
/var/folders/52/y8mfvm1n6vj12ntkl_p9hqw80000gn/T/ipykernel_48711/1253503542.py:64: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"b1 diff: {abs(b1 - model_original.params[1]):.10f}")
/var/folders/52/y8mfvm1n6vj12ntkl_p9hqw80000gn/T/ipykernel_48711/1253503542.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with D